<a href="https://colab.research.google.com/github/shivanswamynathan/NLP/blob/main/Text_summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=7f0874a14ab0fe1b44a668764401d0b4a515b3e69a1709f1b6d3e894e3650c35
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [2]:
import spacy
from transformers import pipeline
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
import urllib.request

In [3]:
nlp = spacy.load("en_core_web_sm")
url = "https://raw.githubusercontent.com/wjbmattingly/freecodecamp_spacy/main/data/wiki_us.txt"
with urllib.request.urlopen(url) as response:
  text = response.read().decode('utf-8')
print(text)

The United States of America (U.S.A. or USA), commonly known as the United States (U.S. or US) or America, is a country primarily located in North America. It consists of 50 states, a federal district, five major unincorporated territories, 326 Indian reservations, and some minor possessions.[j] At 3.8 million square miles (9.8 million square kilometers), it is the world's third- or fourth-largest country by total area.[d] The United States shares significant land borders with Canada to the north and Mexico to the south, as well as limited maritime borders with the Bahamas, Cuba, and Russia.[22] With a population of more than 331 million people, it is the third most populous country in the world. The national capital is Washington, D.C., and the most populous city is New York.

Paleo-Indians migrated from Siberia to the North American mainland at least 12,000 years ago, and European colonization began in the 16th century. The United States emerged from the thirteen British colonies est

In [9]:
text

"The United States of America (U.S.A. or USA), commonly known as the United States (U.S. or US) or America, is a country primarily located in North America. It consists of 50 states, a federal district, five major unincorporated territories, 326 Indian reservations, and some minor possessions.[j] At 3.8 million square miles (9.8 million square kilometers), it is the world's third- or fourth-largest country by total area.[d] The United States shares significant land borders with Canada to the north and Mexico to the south, as well as limited maritime borders with the Bahamas, Cuba, and Russia.[22] With a population of more than 331 million people, it is the third most populous country in the world. The national capital is Washington, D.C., and the most populous city is New York.\n\nPaleo-Indians migrated from Siberia to the North American mainland at least 12,000 years ago, and European colonization began in the 16th century. The United States emerged from the thirteen British colonies 

In [4]:
#Preprocess text using SpaCy.
doc = nlp(text)
processed_text = " ".join([token.lemma_ for token in doc if not token.is_stop and token.is_alpha])
processed_text


'United States America USA commonly know United States America country primarily locate North America consist state federal district major unincorporated territory indian reservation minor million square mile million square kilometer world fourth large country total United States share significant land border Canada north Mexico south limited maritime border Bahamas Cuba population million people populous country world national capital Washington populous city New York Paleo Indians migrate Siberia north american mainland year ago european colonization begin century United States emerge thirteen british colony establish East Coast dispute taxation political representation Great Britain lead American Revolutionary War establish independence late century begin expand North America gradually obtain new territory war frequently displace Native Americans admit new state United States span continent slavery legal southern United States second half century American Civil War lead abolition sp

In [10]:
#Summarization using Hugging Face (BERT-based model)
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
summary = summarizer(text, max_length=100, min_length=30, do_sample=False)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [14]:
summary[0]['summary_text']

" The United States consists of 50 states, a federal district, five major unincorporated territories, 326 Indian reservations, and some minor possessions . At 3.8 million square miles, it is the world's third- or fourth-largest country by total area . The U.S. is a highly developed country, accounts for approximately a quarter of global GDP, and is world's largest economy . The national capital is Washington, D.C., and the most populous city is New York"

In [17]:
#Evaluate summary
# ROUGE evaluation
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(text, summary[0]['summary_text'])
print("\nROUGE Scores:", rouge_scores)


ROUGE Scores: {'rouge1': Score(precision=1.0, recall=0.13368055555555555, fmeasure=0.2358346094946401), 'rouge2': Score(precision=0.9078947368421053, recall=0.12, fmeasure=0.2119815668202765), 'rougeL': Score(precision=0.8831168831168831, recall=0.11805555555555555, fmeasure=0.2082695252679939)}


In [18]:
print("ROUGE Scores:")
for rouge_type, score in rouge_scores.items():
    print(f"  {rouge_type}:")
    print(f"    Precision: {score.precision}")
    print(f"    Recall: {score.recall}")
    print(f"    F-measure: {score.fmeasure}")

ROUGE Scores:
  rouge1:
    Precision: 1.0
    Recall: 0.13368055555555555
    F-measure: 0.2358346094946401
  rouge2:
    Precision: 0.9078947368421053
    Recall: 0.12
    F-measure: 0.2119815668202765
  rougeL:
    Precision: 0.8831168831168831
    Recall: 0.11805555555555555
    F-measure: 0.2082695252679939


In [20]:
#BLEU score
reference = text.split()  # Reference text tokenized
candidate = summary[0]['summary_text'].split()  # Summarized text tokenized
bleu_score = sentence_bleu([reference], candidate)
print(f"BLEU Score: {bleu_score}")

BLEU Score: 0.0011901565775596593
